In [1]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential



In [ ]:
data_dir='\\ML_jupyter\\facemask detector\\dataset\\facemask'
data_dir


In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

In [ ]:
list(data_dir.glob('*/*.jpg'))[:5]

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
with_mask= list(data_dir.glob('with_mask/*'))
with_mask[:5]

In [ ]:
import PIL
PIL.Image.open(str(with_mask[1]))


In [ ]:
without_mask= list(data_dir.glob('without_mask/*'))
without_mask[:5]

In [ ]:
import PIL
PIL.Image.open(str(without_mask[1]))


In [ ]:
face_images_dict = {
    'with_mask': list(data_dir.glob('with_mask/*')),
    'without_mask': list(data_dir.glob('without_mask/*'))}

In [ ]:
face_label_dict={
    'with_mask':0,
    'without_mask':1,
}

In [ ]:
face_images_dict['with_mask'][:5]

In [ ]:
str(face_images_dict['with_mask'][0])

In [ ]:
img = cv2.imread(str(face_images_dict['with_mask'][0]))

In [ ]:
cv2.resize(img,(224,224)).shape

In [ ]:
x=[]
y=[]
for state,images in face_images_dict.items():
    for image in images:
        try:
            img=cv2.imread(str(image))
            resized_image=cv2.resize(img,(224,224),cv2.INTER_AREA)
        except:
            break
        x.append(resized_image)
        y.append(face_label_dict[state])    


In [ ]:
x = np.array(x)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [ ]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [ ]:

feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [ ]:
num_of_states = 2

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_states)
])

model.summary()

In [ ]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=20)

In [ ]:
model.evaluate(X_test_scaled,y_test)

In [ ]:
y_test


In [ ]:
plt.axis('off')
plt.imshow(X_test_scaled[0])

In [ ]:
plt.axis('off')
plt.imshow(X_test_scaled[1])

In [ ]:
plt.axis('off')
plt.imshow(X_test_scaled[2])

In [ ]:
plt.axis('off')
plt.imshow(X_test_scaled[3])

In [ ]:
plt.axis('off')
plt.imshow(X_test_scaled[4])

In [ ]:
y_test[:5]

In [ ]:
model.save("my_detector.pb", save_format="h5")

In [ ]:
tf.keras.experimental.export_saved_model(model, 'path_to_my_model.h5')

In [ ]:
reloaded_model = tf.keras.experimental.load_from_saved_model('path_to_my_model.h5', custom_objects={'KerasLayer':hub.KerasLayer})
print(reloaded_model.get_config())

#Get input shape from model.get_config()
reloaded_model.build((None, 224, 224, 3))
print(reloaded_model.summary())

In [ ]:
model.save('saved.h5')

In [ ]:
new_model = tf.keras.models.load_model('saved.h5',custom_objects={'KerasLayer':hub.KerasLayer})
new_model.summary()